In [ ]:
import pandas as pd
from index import cot_all

In [ ]:
df = cot_all()

> pick the asset you want

In [ ]:
ASSET_CODE = '088691'

# df[(df['Open Interest (Other)']!= 0) & (df['CFTC Contract Market Code'] == ASSET_CODE)]

df = df[df['CFTC Contract Market Code'] == ASSET_CODE]

In [ ]:
df = df.rename(columns={'As of Date in Form YYYY-MM-DD': 'Date'})

df['Date'] = pd.to_datetime(df['Date'])

df.set_index('Date', inplace=True)

df.sort_index(inplace=True)

df.head()

## Net Positions

---

> Commercial (Long) = Producer/Merchant/Processor/User(Long) + Swap Dealers(Long) + Swap Dealers(Spreading)

> Commercial (Short) = Producer/Merchant/Processor/User(Short) + Swap Dealers(Short) + Swap Dealers(Spreading)

> Non-Commercial (Long) = Managed Money(Long) + Other Reportables(Long) 

> Non-Commercial (Short) = Managed Money(Short) + Other Reportables(Short) 

> Non-Commercial (Spreading) = Managed Money(Spreading) + Other Reportables(Spreading) 

In [ ]:
df['Commercial_Net_Position'] = df['Commercial Positions-Long (All)'] - df['Commercial Positions-Short (All)']
df['Large_Speculators_Net_Position'] = df['Noncommercial Positions-Long (All)'] - df['Noncommercial Positions-Short (All)']
df['Small_Traders_Net_Position'] = df['Nonreportable Positions-Long (All)'] - df['Nonreportable Positions-Short (All)']

df[['Commercial_Net_Position', 'Large_Speculators_Net_Position', 'Small_Traders_Net_Position']].head()

## COT Index

The Commitments of Traders (COT) Index is calculated as:

$$
\text{COT Index} = \left( \frac{P_{\text{current}} - P_{\min}}{P_{\max} - P_{\min}} \right) \times 100
$$

Where:

$ P_{\text{current}} $: Current Net Position <br>
$ P_{\min} $: Lowest Net Position in the past  N  period <br>
$ P_{\max} $: Highest Net Position in the past  N  periods <br>
$ N $: Number of periods used in the lookback window


| Strategy                         | Recommended COT Index Period (N) | Notes                                                           |
| -------------------------------- | -------------------------------- | --------------------------------------------------------------- |
| **Swing Trading**                | **26 weeks** (6 months)          | Most popular. Balances short-term moves and sentiment extremes. |
| **Position Trading / Long-Term** | **52 weeks** (1 year)            | Captures major sentiment extremes and long-term reversals.      |
| **Short-Term Trading**           | **13 weeks** (3 months)          | More sensitive to recent changes, but can be noisy.             |
| **Contrarian Approach**          | Use 26–52 weeks                  | To spot positioning at sentiment extremes (close to 0 or 100).  |

In [ ]:
window = 26 # Rolling window for the last N weeks

df['Max_Net_Position_Last_N'] = df['Commercial_Net_Position'].rolling(window=window).max()

df['Min_Net_Position_Last_N'] = df['Commercial_Net_Position'].rolling(window=window).min()

# df.dropna(inplace=True)

df['COT_Index'] = 100 * (df['Commercial_Net_Position'] - df['Min_Net_Position_Last_N'])/(df['Max_Net_Position_Last_N'] - df['Min_Net_Position_Last_N'])

df[['Commercial_Net_Position', 'Large_Speculators_Net_Position', 'Small_Traders_Net_Position', 'COT_Index']].tail()

## Filter by time

In [ ]:
df = df[['Commercial_Net_Position', 'Large_Speculators_Net_Position', 'Small_Traders_Net_Position', 'COT_Index', 'Open Interest (All)']]
df = df.rename(columns={'Open Interest (All)': 'OI'})
df = df[df.index > '2020']

df.head()

## Plotting

### COT index

In [ ]:
import plotly.express as px

fig = px.line(df.reset_index(), x='Date', y='COT_Index', title='COT index', markers=True, template='plotly_dark')

# Add green shaded area (e.g. for high index range)
fig.add_shape(
    type='rect',
    xref='paper',  # entire x-axis
    yref='y',
    x0=0,
    x1=1,
    y0=80,
    y1=100,
    fillcolor='green',
    opacity=0.2,
    line_width=0,
    layer='below'
)

# Add red shaded area (e.g. for low index range)
fig.add_shape(
    type='rect',
    xref='paper',
    yref='y',
    x0=0,
    x1=1,
    y0=0,
    y1=20,
    fillcolor='red',
    opacity=0.2,
    line_width=0,
    layer='below'
)

fig.write_html("plots/cot_index.html")

fig.show()


### Open Interest

In [ ]:
import plotly.express as px

fig = px.line(df.reset_index(), x='Date', y='OI', title='Open Interest', markers=True, template='plotly_dark')

fig.write_html("plots/open_interest.html")

fig.show()

### Net positions

In [ ]:
import plotly.graph_objects as go

fig = go.Figure()

for col in ['Commercial_Net_Position', 'Large_Speculators_Net_Position', 'Small_Traders_Net_Position']:
    fig.add_trace(go.Scatter(x=df.index, y=df[col], mode='lines', name=col.replace('_Net_Position', '').replace('_', ' ').title(), line=dict(width=2)))

fig.update_layout(
        title='Net positions',
        xaxis_title='Date',
        yaxis_title='Contratcs',
        template='plotly_dark',
        height=600,
        width=1500,
    )

fig.write_html("plots/net_positions.html")
fig.show()

### Plotting all charts


In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Create subplots: 1 column, 3 rows, shared X-axis
fig = make_subplots(rows=3, cols=1, shared_xaxes=True, vertical_spacing= 0.1, subplot_titles=('COT Index', 'Net positions', 'OI'))

# Add each series to its own subplot
fig.add_trace(go.Scatter(x=df.index, y=df['COT_Index'], name='COT Index'), row=1, col=1)
for col in ['Commercial_Net_Position', 'Large_Speculators_Net_Position', 'Small_Traders_Net_Position']:
    fig.add_trace(go.Scatter(x=df.index, y=df[col], name=col.replace('_Net_Position', '').replace('_', ' ').title(), line=dict(width=2)), row=2, col=1)
fig.add_trace(go.Scatter(x=df.index, y=df['OI'], name='OI'), row=3, col=1)

fig.add_shape(
    type='rect',
    xref='paper',  # spans the full width of the plot
    yref='y1',  # y-axis of subplot in row 1
    x0=0,
    x1=1,
    y0=80,       # Adjust based on your y-axis range
    y1=100,
    fillcolor='green',
    opacity=0.2,
    line_width=0,
    layer='below'
)

fig.add_shape(
    type='rect',
    xref='paper',  # spans the full width of the plot
    yref='y1',  # y-axis of subplot in row 1
    x0=0,
    x1=1,
    y0=0,              # Adjust based on your y-axis range
    y1=20,
    fillcolor='red',
    opacity=0.2,
    line_width=0,
    layer='below'
)


# Update layout
fig.update_layout(
    height=900,
    width=1450,
    title_text='Multiple Time Series with Shared X-Axis',
    template='plotly_dark',
    hovermode='x unified',  # or 'x unified' or 'closest'
)
fig.update_xaxes(title_text='Date', row=3, col=1)  # Only show x-axis label on bottom chart
fig.update_yaxes(title_text='CONTRACT')

# Save as responsive HTML
fig.write_html("plots/cot_analysis.html", auto_open=False, full_html=True, include_plotlyjs='cdn', config={'responsive': True})
fig.show()